In [1]:
ON_COLAB = True

if ON_COLAB:
    
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# torch
import torch
from torch.nn import Conv2d, MaxPool2d
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import sampler
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
# other
import sys
sys.path.append('drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/')
import params
import matplotlib.pyplot as plt
import numpy as np
import os 
from pathlib import Path
import pandas as pd
import imageio
from csv import writer
from csv import reader
from PIL import Image
!pip install trimesh
import trimesh
!pip install pyrender
import pyrender
if ON_COLAB:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
!pip install wandb
import wandb
!wandb login --relogin
# api_key = f2c3bd208d418cf11dcc2c2cbf5e4bc0fc104421

# set the seed for reproducibility
rng_seed = 90
torch.manual_seed(rng_seed)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
par = params.Parameters()
print(par.epochs)
print(par.resume)
print(par.run_id)
print(par.batch_size)
print(par.learning_rate)
print(par.decay)


150
True
1mvukpct
128
0.001
0.0001


In [4]:
os.environ["PYOPENGL_PLATFORM"] = "egl" #opengl seems to only work with TPU
!PYOPENGL_PLATFORM=egl python -c "from OpenGL import EGL"
print(os.environ['PYOPENGL_PLATFORM']) 

import OpenGL.GL as gl
print(gl.glGetString(gl.GL_VERSION))
print(gl.glGetString(gl.GL_VENDOR)) 


egl
None
None


In [ ]:
#txt_path = par.txt_path
txt_path = 'drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/poses_45_DLC_txt'

def get_nth_line(fobj, n):
    for i in range(n):
        next(fobj)
    return next(fobj)

columns = ['id', 'x', 'y', 'z', 'roll', 'pitch', 'yaw']

with open('drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/poses_45_DLC.csv', 'w') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(columns)
    for f in Path(txt_path).iterdir():
        id = int(str(f).split('/')[-1].split('.txt')[0]) 
        file = open(f)
        line4 = get_nth_line(file, 4).split('\n')[0]
        line_splitted = line4.split(' ')
        writer_object.writerow([id] + line_splitted)
    f_object.close()

In [8]:
data = np.loadtxt('drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/poses_45_DLC.csv', delimiter = ',', skiprows=1)
print('data', data[0:2,4:])
data_cos = np.cos(np.deg2rad(data[:, 4:]))
data_sin = np.sin(np.deg2rad(data[: ,4:]))
print('data_cos', data_cos.shape)
print('data_sin', data_sin)

data_cos_roll = list(data_cos[:,0])
data_sin_roll = list(data_sin[:,0])
data_cos_pitch = list(data_cos[:,1])
data_sin_pitch = list(data_sin[:,1])
data_cos_yaw = list(data_cos[:,2])
data_sin_yaw = list(data_sin[:,2])

data = np.delete(data, 0, axis=1)
data = np.delete(data, 3, axis=1)
data = np.delete(data, 3, axis=1)
data = np.delete(data, 3, axis=1)
print('data', data)
data = np.append(data, np.reshape(np.array(data_cos_roll), (-1, 1)), axis = 1)
data = np.append(data, np.reshape(np.array(data_sin_roll), (-1, 1)), axis = 1)
data = np.append(data, np.reshape(np.array(data_cos_pitch), (-1, 1)), axis = 1)
data = np.append(data, np.reshape(np.array(data_sin_pitch), (-1, 1)), axis = 1)
data = np.append(data, np.reshape(np.array(data_cos_yaw), (-1, 1)), axis = 1)
data = np.append(data, np.reshape(np.array(data_sin_yaw), (-1, 1)), axis = 1)
print('data', data)


data_sum = np.sum(data, axis=0)
print('sum',data_sum.shape)
data_mean = data_sum/data.shape[0]
print('mean', data_mean)
std = np.std(data, axis = 0)
print('std', std)

data [[180.        46.678375 -96.840393]
 [180.        14.970141 125.132668]]
data_cos (19991, 3)
data_sin [[ 1.22464680e-16  7.27513859e-01 -9.92881788e-01]
 [ 1.22464680e-16  2.58315629e-01  8.17821737e-01]
 [-1.22464680e-16  3.06833372e-01 -9.65196358e-01]
 ...
 [ 8.02851456e-07  9.55700767e-01  8.43531219e-01]
 [-1.22464680e-16  3.04393232e-01  8.31372169e-01]
 [ 1.22464680e-16  2.36034401e-01  5.59139033e-01]]
data [[   8.  -45.  -92.]
 [ -45.   32. -398.]
 [  -5.  -39. -292.]
 ...
 [ -57.   60.  -42.]
 [  54.   53. -397.]
 [  20.   35. -373.]]
data [[ 8.00000000e+00 -4.50000000e+01 -9.20000000e+01 ...  7.27513859e-01
  -1.19103970e-01 -9.92881788e-01]
 [-4.50000000e+01  3.20000000e+01 -3.98000000e+02 ...  2.58315629e-01
  -5.75471638e-01  8.17821737e-01]
 [-5.00000000e+00 -3.90000000e+01 -2.92000000e+02 ...  3.06833372e-01
  -2.61526272e-01 -9.65196358e-01]
 ...
 [-5.70000000e+01  6.00000000e+01 -4.20000000e+01 ...  9.55700767e-01
  -5.37080145e-01  8.43531219e-01]
 [ 5.40000000e

In [9]:
class PoseDataset(Dataset):

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        poses = pd.read_csv(csv_file)
        self.poses = poses#.sort_values(by=["id"])
        print(self.poses)
        keep_col = ['id','x','y','z']
        self.poses = self.poses[keep_col]
        print('poses', self.poses)
        self.poses['cos_roll'] = data_cos_roll
        self.poses['sin_roll'] = data_sin_roll
        self.poses['cos_pitch'] = data_cos_pitch
        self.poses['sin_pitch'] = data_sin_pitch
        self.poses['cos_yaw'] = data_cos_yaw
        self.poses['sin_yaw'] = data_sin_yaw
        print('poses', self.poses)
        for i in range(1,4):
            #print(mean[i-1])
            #print(self.poses.iloc[:, i])
            self.poses.iloc[:, i] = self.poses.iloc[:,i].subtract(data_mean[i-1])
            #print(self.poses)
            self.poses.iloc[:, i] = self.poses.iloc[:,i].divide(std[i-1])
        print('poses', self.poses)
        self.root_dir = root_dir
  

    def __len__(self):
        return len(self.poses)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.root_dir,
                                str(self.poses.iloc[idx, 0]) + '-N_.png')
        
        img = imageio.imread(img_path, ignoregamma = True)
        rgb_img = img[:,:,:3]
        img_tensor = torch.tensor(rgb_img)/255.0
        img_tensor = np.transpose(img_tensor, (2, 0, 1))
        
        img_downsampled = F.interpolate(img_tensor, scale_factor = 0.25)
        img_downsampled = np.transpose(img_downsampled, (0, 2, 1))
        img_downsampled = F.interpolate(img_downsampled, scale_factor = 0.25)
        img_downsampled = np.transpose(img_downsampled, (0, 2, 1))

        pose = self.poses.iloc[idx, 1:]
        pose = np.array([pose])
        pose =  pose.reshape(9)
        return img_downsampled, pose

In [ ]:
class PoseDataset(Dataset):

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        poses = pd.read_csv(csv_file)
        self.poses = poses.sort_values(by=["id"])
        self.root_dir = root_dir
  

    def __len__(self):
        return len(self.poses)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        #print('idx', idx)
        img_path = os.path.join(self.root_dir,
                                str(self.poses.iloc[idx, 0]) + '-N_.png')
        #print('1:', str(self.poses))
        #print('2:', str(self.poses.iloc[idx]))
        #print('3:', str(self.poses.iloc[idx, 0]) + '-N_.png')
        img = imageio.imread(img_path, ignoregamma = True)
        #print('image', img.shape)
        rgb_img = img[:,:,:3]
        #print('rgb_img', rgb_img.shape)
        '''figure = plt.figure(figsize = (10, 10))
        plt.imshow(rgb_img)
        plt.show() '''
        #print('type_rgb', type(rgb_img))
        img_tensor = torch.tensor(rgb_img)/255.0
        img_tensor = np.transpose(img_tensor, (2, 0, 1))
        #img_tensor = img_tensor.permute(2, 0, 1)
        
        #print('img_tensor', img_tensor.shape)
        img_downsampled = F.interpolate(img_tensor, scale_factor = 0.25)
        #print('img_downsampled', img_downsampled.shape)
        img_downsampled = np.transpose(img_downsampled, (0, 2, 1))
        #print('img_downsampled', img_downsampled.shape)
        img_downsampled = F.interpolate(img_downsampled, scale_factor = 0.25)
        #print(img_downsampled.shape)
        img_downsampled = np.transpose(img_downsampled, (2, 1, 0))
        #print('img_downsampled',img_downsampled.shape)
        '''figure = plt.figure(figsize = (10, 10))
        plt.imshow(img_downsampled)
        plt.show() '''

        pose = self.poses.iloc[idx, 1:]
        #print('pose', type(pose))
        pose = np.array([pose])
        pose =  pose.reshape(6)
        return img_downsampled, pose

In [11]:
pose_dataset = PoseDataset(csv_file = 'drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/poses_45_DLC.csv',
                           root_dir='drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/dataset_45_DLC')
#print(len(pose_dataset))

          id     x     y      z        roll      pitch         yaw
0      18991   8.0 -45.0  -92.0  180.000000  46.678375  -96.840393
1      18992 -45.0  32.0 -398.0  180.000000  14.970141  125.132668
2      18993  -5.0 -39.0 -292.0 -180.000000  17.868498 -105.160645
3      18994 -57.0 -61.0  -92.0  180.000000  56.398033 -139.751083
4      18995   9.0 -14.0  -33.0  179.999985  65.012375  -32.390202
...      ...   ...   ...    ...         ...        ...         ...
19986    986  58.0  32.0  -57.0  179.999985  65.090782   22.846962
19987    987 -12.0  48.0 -184.0  180.000000   9.821634 -173.039978
19988    988 -57.0  60.0  -42.0  179.999954  72.882034  122.485092
19989    989  54.0  53.0 -397.0 -180.000000  17.721663   56.239952
19990    990  20.0  35.0 -373.0  180.000000  13.652605   33.996277

[19991 rows x 7 columns]
poses           id     x     y      z
0      18991   8.0 -45.0  -92.0
1      18992 -45.0  32.0 -398.0
2      18993  -5.0 -39.0 -292.0
3      18994 -57.0 -61.0  -92.0
4   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
'''transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize(mean.tolist(), std.tolist()),
    ]
)'''

# Create train val test split
n = len(pose_dataset)
n_1 = int(n/8)

train_set, val_set, test_set = torch.utils.data.random_split(
    pose_dataset, [n-(2*n_1), n_1, n_1])

print(len(train_set), len(val_set), len(test_set))

loader_train = DataLoader(train_set, batch_size=par.batch_size, shuffle=par.shuffle, num_workers=2)
loader_val = DataLoader(val_set, batch_size=par.batch_size, shuffle=par.shuffle, num_workers=2)
loader_test = DataLoader(test_set, batch_size=par.batch_size, shuffle=par.shuffle, num_workers=2)

14995 2498 2498


In [ ]:
### TO DO ### UNDERSTAND IF THE MEAN AND STD SHOULD BE DIFFERENT WRT THE OTHER EXAMPLE AND IF YES HOW

unnormalize = transforms.Normalize(
    (-mean / std).tolist(), (1.0 / std).tolist())

def denorm(x):
    '''
    Function to reverse the normalization so that we can visualise the outputs
    '''
    x = unnormalize(x)
    x = x.view(x.size(0), 3, 256, 256)
    return x

def show(img):
    '''
    function to visualise tensors
    '''
    if torch.cuda.is_available():
        img = img.cpu()
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)).clip(0, 1))

In [13]:
# define Resnet building blocks
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(Conv2d(inchannel, outchannel, kernel_size=3,
                                         stride=stride, padding=1, bias=False),
                                  nn.BatchNorm2d(outchannel),
                                  nn.ReLU(inplace=True),
                                  Conv2d(outchannel, outchannel, kernel_size=3,
                                         stride=1, padding=1, bias=False),
                                  nn.BatchNorm2d(outchannel))
        self.shortcut = nn.Sequential()

        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(Conv2d(inchannel, outchannel,
                                                 kernel_size=1, stride=stride,
                                                 padding=0, bias=False),
                                          nn.BatchNorm2d(outchannel))

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        print(out.shape)
        out = F.relu(out)
        print(out.shape)
        return out

# define Resnet
class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=9):
        super(ResNet, self).__init__()

        self.inchannel = 64
        self.conv1 = nn.Sequential(Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3, bias = False), 
                                  nn.BatchNorm2d(64), 
                                  nn.ReLU())
        self.maxpool = nn.MaxPool2d(2, stride=2, padding=1)
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride = 2)
        self.maxpool2 = nn.MaxPool2d(2, stride=1, padding=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride = 2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride = 2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride = 2)
        self.layer5 = self.make_layer(ResidualBlock, 256, 2, stride = 2)
        self.avgpool = nn.AvgPool2d(4)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        #print('x_shape', x.shape)
        x = self.conv1(x)
        #print('x_shape', x.shape)
        x = self.maxpool(x)
        #print('x_shape', x.shape)
        x = self.layer1(x)
        #print('x_shape', x.shape)
        x = self.maxpool2(x)
        #print('x_shape', x.shape)
        x = self.layer2(x)
        #print('x_shape', x.shape)
        x = self.maxpool2(x)
        #print('x_shape', x.shape)
        x = self.layer3(x)
        #print('x_shape', x.shape)
        x = self.maxpool2(x)
        #print('x_shape', x.shape)
        x = self.layer4(x)
        #print('x_shape', x.shape)
        #x = self.maxpool2(x)
        #print('x_shape', x.shape)
        #x = self.layer4(x)
        #print('x_shape', x.shape)
        x = self.avgpool(x)
        #print('x_shape', x.shape)
        x = x.view(x.size(0), -1)
        #print('x_shape', x.shape)
        x = self.fc(x)
        #print('x_shape', x.shape)
        x[:,3:9] = torch.tanh(x[:,3:9])
        return x

def MyResNet():
    return ResNet(ResidualBlock)

In [10]:
dtype = torch.float32

if par.USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
print(device)

print_every = par.print_every

def train_part(model, optimizer, epochs, start_e):
    """
    Train a model on NaturalImageNet using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.
    """

    iter_val = iter(loader_val)
    losses = []
    val_losses = []

    tra_losses = []
    tra_val_losses = []

    rot_losses = []
    rot_val_losses = []

    for e in tqdm(range(start_e, epochs)):
        if e == 0:
            learning_rate = par.learning_rate
        else:
            learning_rate = learning_rate * 1/(1 + par.decay * e)
        optimizer = optim.Adamax(model.parameters(), lr=learning_rate, weight_decay=par.weight_decay)
        for t, (x, y) in enumerate(tqdm(loader_train, leave=False)):
            
            #print('learning_rate', learning_rate)
            #print("x shape", x.shape)
            try:
                x_val, y_val = next(iter_val)
            except StopIteration:
                iter_val = iter(loader_val)
                x_val, y_val = next(iter_val)

            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            #print('x', x)
            y = y.to(device=device, dtype=torch.float32)
            #print('y', y)
            y_tra = y[:, :3]
            #print('y_tra', y_tra)
            y_rot = y[:, 3:]
            #print('y_rot', y_rot)

            x_val = x_val.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y_val = y_val.to(device=device, dtype=dtype)
            #print('y_val', y_val)
            y_val_tra = y_val[:, :3]
            #print('y_val_tra', y_val_tra)
            y_val_rot = y_val[:, 3:]
            #print('y_val_rot', y_val_rot)

            scores = model(x)
            #print('scores', (scores))
            scores_tra = scores[:, :3]
            #print('scores_tra', scores_tra)
            scores_rot = scores[:, 3:]
            #print('scores_rot', scores_rot)

            loss = F.mse_loss(scores, y)
            loss_tra = F.mse_loss(scores_tra, y_tra)
            loss_rot = F.mse_loss(scores_rot, y_rot)
            #print('loss', loss)
            losses.append(loss.item())
            tra_losses.append(loss_tra.item())
            rot_losses.append(loss_rot.item())

            scores_val = model(x_val)
            #print('scores_val', scores_val)
            scores_val_tra = scores_val[:, :3]
            #print('scores_val_tra', scores_val_tra)
            scores_val_rot = scores_val[:, 3:]
            #print('scores_val_rot', scores_val_rot)

            val_loss = F.mse_loss(scores_val, y_val)
            val_loss_tra = F.mse_loss(scores_val_tra, y_val_tra)
            val_loss_rot = F.mse_loss(scores_val_rot, y_val_rot)
            #print('val_loss', loss)
            val_losses.append(val_loss.item())
            tra_val_losses.append(val_loss_tra.item())
            rot_val_losses.append(val_loss_rot.item())

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            loss.backward()

            wandb.log({
                "loss": loss,
                "val_loss": val_loss,
                "tra_loss [cm]": loss_tra,
                "tra_val_loss [cm]": val_loss_tra,
                "rot_loss [°]": loss_rot,
                "rot_val_loss [°]": val_loss_rot,
                "learning_rate": learning_rate
                })
        

            # Update the parameters of the model using the gradients
            optimizer.step()

            if t % print_every == 0:
                print('Epoch: %d, Iteration %d, loss = %.4f' % (e, t, loss_tra.item()))
                print('Epoch: %d, Iteration %d, loss = %.4f' % (e, t, loss_rot.item()))

        torch.save({ # Save our checkpoint loc
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, f"{par.checkpoint_path}/{run.id}.tar")
        wandb.save(f"{par.checkpoint_path}/{run.id}.tar") # saves checkpoint to wandb

    plt.figure(figsize=(10,7))
    plt.title("Learning curve traslations")
    plt.plot(tra_losses, label="Training loss")
    plt.plot(tra_val_losses, label="Validation loss")
    plt.ylabel('Loss')
    plt.xlabel('Iterations')
    plt.legend()
    plt.savefig('learning_curve.png', bbox_inches='tight', dpi=150)
    plt.show()

    plt.figure(figsize=(10,7))
    plt.title("Learning curve rotations")
    plt.plot(rot_losses, label="Training loss")
    plt.plot(rot_val_losses, label="Validation loss")
    plt.ylabel('Loss')
    plt.xlabel('Iterations')
    plt.legend()
    plt.savefig('learning_curve.png', bbox_inches='tight', dpi=150)
    plt.show()


cuda:0


In [14]:
print(par.run_id)
if par.resume:
    run = wandb.init(project="Deep Pose Estimation", 
                        entity="cuccomat",
                        id=par.run_id,
                        resume=True)
else:
    run = wandb.init(project="Deep Pose Estimation", 
            entity="cuccomat",
            config = {
                    "learning_rate": par.learning_rate,
                    "lr_decay": par.decay,
                    "weight_decay": par.weight_decay,
                    "batch_size": par.batch_size,
                    "epochs": par.epochs
                })

1mvukpct


wandb: Currently logged in as: cuccomat. Use `wandb login --relogin` to force relogin
wandb: WARNING Tried to auto resume run with id 2ab33ph8 but id 1mvukpct is set.


In [15]:
print(torch.cuda.is_available())
if par.USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
print(device)

torch.autograd.set_detect_anomaly(True)
model = MyResNet()
model = model.to(device=device)  # move the model parameters to CPU/GPU
optimizer = optim.Adamax(model.parameters(), lr=par.learning_rate, weight_decay=par.weight_decay)
epoch = 0
if wandb.run.resumed:
    wandb.restore(f"{par.checkpoint_path}/{par.run_id}.tar")
    checkpoint = torch.load(f"{par.checkpoint_path}/{par.run_id}.tar")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    print(epoch)

True
cuda:0
89


In [ ]:
# define and train the network
'''params_sum = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params_sum))'''

train_part(model, optimizer, epochs=par.epochs, start_e=epoch)

losses = []
val_losses = []

# save the model
torch.save(model.state_dict(), 'model.pt')

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
Epoch: 89, Iteration 0, loss = 0.9463
Epoch: 89,

  0%|          | 0/118 [00:00<?, ?it/s]

torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
Epoch: 90, Iteration 0, loss = 0.7170
Epoch: 90,

...

torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
Epoch: 93, Iteration 0, loss = 0.3671
Epoch: 93,

  0%|          | 0/118 [00:00<?, ?it/s]

torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
Epoch: 94, Iteration 0, loss = 0.3811
Epoch: 94,

  0%|          | 0/118 [00:00<?, ?it/s]

torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
Epoch: 95, Iteration 0, loss = 0.3785
Epoch: 95,

In [28]:
 from sklearn.neighbors import KDTree
diameter = 120
def compute_adds_score(pts3d, pose_gt, pose_pred, percentage=0.1):
    #print('pose_gt', pose_gt)
    #print('pose_pred', pose_pred)
    #print('pose_gt[0]', type(pose_gt[0]))
    #print('pose_gt[1]', type(pose_gt[1]))
    #print('pose_pred[0]', type(pose_pred[0]))
    #print('pose_pred[1]', type(pose_pred[1]))
    R_gt = pose_gt[0].numpy()
    t_gt= pose_gt[1].numpy()
    R_pred = pose_pred[0].numpy()
    t_pred = pose_pred[1].numpy()

    count = R_gt.shape[0]
    #print('count', count)
    mean_distances = np.zeros((count,), dtype=np.float32)
    for i in range(count):
        #print('pred')
        #print(t_pred[i])
        #print(t_pred[i].shape)
        #print(type(t_pred[i]))
        if np.isnan(np.sum(t_pred[i])):
            mean_distances[i] = np.inf
            continue
        #print('R_gt[i]', R_gt[i].shape)
        #print('pts3d', pts3d.shape)
        #print('pts3d.transpose()', pts3d.transpose().shape)
        #print('t_gt[i]', t_gt[i].shape)
        #print('t_gt[i]', t_gt[i])
        pts_xformed_gt = R_gt[i] * pts3d.transpose() + t_gt[i][:, None]#np.squeeze(t_gt[i], axis=1)
        #print('pts_xformed_gt', pts_xformed_gt.shape)
        pts_xformed_pred = R_pred[i] * pts3d.transpose() + t_pred[i][:, None]#np.squeeze(t_pred[i], axis=1)
        #print('pts_xformed_pred', pts_xformed_pred.shape)
        kdt = KDTree(pts_xformed_gt.transpose(), metric='euclidean')
        distance, _ = kdt.query(pts_xformed_pred.transpose(), k=1)
        #print('distance', distance.shape)
        mean_distances[i] = np.mean(distance)
        #print('mean_distances[i]', mean_distances[i])
    #print('mean_distances', mean_distances)
    #print('mean_distances.shape', mean_distances.shape)
    #print('mean_distances_tresh.shape',(mean_distances<1.2).shape)
    threshold = diameter*percentage
    score = (mean_distances<threshold).sum() / count
    return score

In [29]:
def print_pose(model_path, pose_p, pose_gt, img, t):
    #print('img_type', type(img))
    tm = trimesh.load(model_path) 
    mesh = pyrender.Mesh.from_trimesh(tm)
    pose_p = torch.squeeze(pose_p, dim=0) 
    #print('xyz_rpy_p', pose_p.cpu().numpy().shape)
    tx_p, ty_p, tz_p, rx_p, ry_p, rz_p = pose_p.cpu().numpy()
    #print('xyz_rpy_p', pose_p)

    r_p = R.from_euler('xyz',[rx_p, ry_p, rz_p], degrees=True)
    #r = R.from_euler('zyx',[rx, ry, rz], degrees=True)

    Twc_p = np.eye(4)
    Twc_p[:3,:3] = r_p.as_matrix()
    Twc_p[:3,3] = np.array([tx_p, ty_p, tz_p])

    Twc_p [0,1] *= -1
    Twc_p [1,0] *= -1
    Twc_p [1,2] *= -1
    Twc_p [2,1] *= -1

    Twc_p [0,3] *= -1

    pose_gt = torch.squeeze(pose_gt, dim=0) 
    #print('xyz_rpy_gt', pose_gt.cpu().numpy().shape)
    tx_gt, ty_gt, tz_gt, rx_gt, ry_gt, rz_gt = pose_gt.cpu().numpy()
    #print('xyz_rpy_gt', pose_gt)

    r_gt = R.from_euler('xyz',[rx_gt, ry_gt, rz_gt], degrees=True)
    #r = R.from_euler('zyx',[rx, ry, rz], degrees=True)

    Twc_gt = np.eye(4)
    Twc_gt[:3,:3] = r_gt.as_matrix()
    Twc_gt[:3,3] = np.array([tx_gt, ty_gt, tz_gt])

    Twc_gt [0,1] *= -1
    Twc_gt [1,0] *= -1
    Twc_gt [1,2] *= -1
    Twc_gt [2,1] *= -1

    Twc_gt [0,3] *= -1

    #print('Twc', Twc)

    im_width, im_height = (1024,1024)
    camK = np.array([[886.81,0.0,512.0],[0.0,886.81,512.0],[0.0,0.0,1.0]])

    camera = pyrender.IntrinsicsCamera(camK[0,0],camK[1,1],
                                        camK[0,2],camK[1,2], zfar = 500.0)

    scene=pyrender.Scene()
    scene.add(camera,pose=Twc_p)
    scene.add(mesh,pose=np.eye(4))
    r = pyrender.OffscreenRenderer(im_width, im_height)
    color,depth = r.render(scene)

    #print('depth', np.shape(depth))
    #print('dept_shape')

    # Flip y in image space
    color = color[::-1,:,:]
    depth = depth[::-1,:]

    # Flip x in image space
    color = color[:,::-1,:]
    depth = depth[:,::-1]

    # Overlap the two images for comparision
    '''figure = plt.figure(figsize = (5, 5))
    plt.imshow(img, alpha = 0.8)#, cmap = plt.cm.gray)
    plt.imshow(depth, alpha = 0.3, cmap = plt.cm.gray_r)'''
    #print('depth', depth)
    #wandb.log({"Test samples overlapping pose generalised dataset": figure})
    '''plt.axis('off')
    plt.show() ''' 
    #figure.savefig('overlap'+str(t)+'.png', bbox_inches='tight', pad_inches = 0)
    return Twc_p[:3,:3], Twc_p[:3,3], Twc_gt[:3,:3], Twc_gt[:3,3]

In [18]:
std = torch.from_numpy(std)
mean = torch.from_numpy(data_mean)

In [30]:
vertices = pd.read_csv('drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/CAD_model/point_cloud.xyz', delimiter = ' ')
vertices = vertices.to_numpy()
vertices = vertices[:, :3]
print('vertices', vertices)
print('vertices', vertices.shape)
print('vertices', type(vertices))

vertices [[ 51.359047  -4.872973  10.201732]
 [ 36.915321 -44.580494   0.      ]
 [ 48.38636  -28.990963   3.235307]
 ...
 [ -5.125973 -58.79953    0.      ]
 [ 20.060768 -45.923706  10.693056]
 [ 27.104954  46.353596   7.599735]]
vertices (11071, 3)
vertices <class 'numpy.ndarray'>


In [31]:
import time
from scipy.spatial.transform import Rotation as R

# TEST
#Is it possibile to set an initial value for traslations and rotations?
def test_part(model):
    global mean
    global std
    iter_test = iter(loader_test)
    with torch.no_grad():
        score = []
        for t, (x, y) in enumerate(tqdm(loader_test, leave=False)):
            #print('x', x[t])
            #print('y', y[t])
            start_time = time.time()
            try:
                x_test, y_test = next(iter_test)
            except StopIteration:
                iter_test = iter(loader_test)
                x_test, y_test = next(iter_test)

            x_test = x_test.to(device=device, dtype=dtype)
            y_test = y_test.to(device=device, dtype=dtype)


            data_loading_time = time.time()

            predictions = model(x_test)
            model_time = time.time()
            test_loss = loss = F.mse_loss(predictions, y_test)
            wandb.log({"Test loss on new datatset": test_loss})

            #print('predictions', predictions.shape)
            #print('mean', mean)
            #print('std', std)
            std = std.to("cuda")
            mean = mean.to("cuda")

            #print('y_test', y_test)

            y_test[:,:3] = (y_test[:,:3] * std[:3]) + mean[:3]
            y_test_6 = torch.zeros((y_test.shape[0], 6))
            y_test_6[:,:3] =  y_test[:,:3]

            y_test_6[:, 3] = torch.rad2deg(torch.acos(y_test[:, 3])) * torch.sign(y_test[:, 4])
            y_test_6[:, 4] = torch.rad2deg(torch.acos(y_test[:, 5])) * torch.sign(y_test[:, 6])
            y_test_6[:, 5] = torch.rad2deg(torch.acos(y_test[:, 7])) * torch.sign(y_test[:, 8])
            #print('y_test_6', y_test_6.shape)

            predictions[:,:3] = (predictions[:,:3] * std[:3]) + mean[:3]
            predictions_6 = torch.zeros((predictions.shape[0], 6))
            predictions_6[:,:3] =  predictions[:,:3]
            #print('predictions_6', predictions_6)

            predictions_6[:, 3] = torch.rad2deg(torch.acos(predictions[:, 3])) * torch.sign(predictions[:, 4])
            predictions_6[:, 4] = torch.rad2deg(torch.acos(predictions[:, 5])) * torch.sign(predictions[:, 6])
            predictions_6[:, 5] = torch.rad2deg(torch.acos(predictions[:, 7])) * torch.sign(predictions[:, 8])
            #print('predictions_6', predictions_6.shape)
            end_time =time.time()
            print('data_loading_time', data_loading_time-start_time)
            print('model_time', model_time-data_loading_time)
            print('data_loadingtime', end_time-model_time)
            print('total_time', end_time-start_time)

            mean_error = []
            for i in range(6):
                mean_error.append(F.l1_loss(predictions_6[:,i], y_test_6[:,i]))
            #print('mean_error', mean_error)

            for t, xyz_rpy in enumerate(predictions_6):
                transform = transforms.ToPILImage()
                #image = transform(x_test[t, :, :, :])
                x = torch.squeeze(x_test[t, :, :, :], dim=0)
                image_upsampled = F.interpolate(x, scale_factor = 2).cpu()
                image_upsampled = np.transpose(image_upsampled, (0, 2, 1))
                image_upsampled = F.interpolate(image_upsampled, scale_factor = 2)
                image_upsampled = np.transpose(image_upsampled, (0, 2, 1))
                image_upsampled = F.interpolate(image_upsampled, scale_factor = 2)
                image_upsampled = np.transpose(image_upsampled, (0, 2, 1))
                image_upsampled = F.interpolate(image_upsampled, scale_factor = 2)
                image_upsampled = np.transpose(image_upsampled, (0, 2, 1))
                image = transform(image_upsampled)
                R_p, t_p, R_gt, t_gt = print_pose(model_path=par.tm_path, pose_p=xyz_rpy, pose_gt = y_test_6[t], img=image, t=t)
                if t == 0:
                    R_p_tens, t_p_tens, R_gt_tens, t_gt_tens  = R_p[None, :], t_p[None, :], R_gt[None, :], t_gt[None, :]
                else:
                    R_p_tens, t_p_tens, R_gt_tens, t_gt_tens = torch.cat((torch.tensor(R_p_tens), torch.tensor(R_p[None, :]))), torch.cat((torch.tensor(t_p_tens), torch.tensor(t_p[None, :]))), torch.cat((torch.tensor(R_gt_tens), torch.tensor(R_gt[None, :]))), torch.cat((torch.tensor(t_gt_tens), torch.tensor(t_gt[None, :])))
            #print('R_p_tens', R_p_tens.shape)
            #pts3d = read_3d_points()

            pts3d = np.asmatrix(vertices)
            
            #print('pts3d', pts3d.shape)
            #print('pts3d', type(pts3d))
            #print('R_p', R_p.shape)
            #print('R_gt', R_gt.shape)
            #print('t_p', t_p.shape)
            #print('t_gt', t_gt.shape)
            score_pred = compute_adds_score(pts3d,
                        (R_gt_tens, t_gt_tens),
                        (R_p_tens, t_p_tens)) 
            score.append(score_pred)
            print('The score in the' + str(t) + 'batch is: ', score_pred)
            print('The score in the' + str(t) + 'batch in percetange is: ', score_pred*100)
        final_score = sum(score)*100 / len(score)
        print('Final score: ', final_score)
                 

In [32]:
dtype = torch.float32

device = torch.device('cuda')
print(device)

model = MyResNet()
model = model.to(device=device)
checkpoint = torch.load(f"{par.checkpoint_path}/{par.run_id}.tar")
model.load_state_dict(checkpoint['model_state_dict'])

test_part(model)

cuda


  0%|          | 0/20 [00:00<?, ?it/s]

torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
data_loading_time 0.09131002426147461
model_time 0.029016971588134766
data_loadingtime 0.019066572189331055
total_time 0.13939356803894043


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage 

The score in the127batch is:  0.9921875
The score in the127batch in percetange is:  99.21875
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 64, 33, 33])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 128, 17, 17])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 256, 9, 9])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
torch.Size([128, 512, 5, 5])
data_loading_time 0.05037355422973633
model_time 0.031389713287353516
data_loadingtime 0.016369342803955078
total_time 0.09813261032104492


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage 

The score in the127batch is:  0.9921875
The score in the127batch in percetange is:  99.21875
torch.Size([66, 64, 33, 33])
torch.Size([66, 64, 33, 33])
torch.Size([66, 64, 33, 33])
torch.Size([66, 64, 33, 33])
torch.Size([66, 128, 17, 17])
torch.Size([66, 128, 17, 17])
torch.Size([66, 128, 17, 17])
torch.Size([66, 128, 17, 17])
torch.Size([66, 256, 9, 9])
torch.Size([66, 256, 9, 9])
torch.Size([66, 256, 9, 9])
torch.Size([66, 256, 9, 9])
torch.Size([66, 512, 5, 5])
torch.Size([66, 512, 5, 5])
torch.Size([66, 512, 5, 5])
torch.Size([66, 512, 5, 5])
data_loading_time 0.021162986755371094
model_time 0.023334026336669922
data_loadingtime 0.0055425167083740234
total_time 0.05003952980041504


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage 

The score in the65batch is:  1.0
The score in the65batch in percetange is:  100.0
Final score:  99.4140625


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn